In [117]:
# import libraries
import os
import pandas as pd
import numpy as np

In [118]:
# fasta reader
def read_fasta(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines


In [119]:
# fasta parser
def parse_fasta(lines):
    seq_list = []
    seq=''
    for line,index in zip(lines,range(len(lines))):
        if index == len(lines)-1:
            seq_list.append(seq)
        if line.startswith('>'):
            seq_list.append(seq)
            seq = ''
            continue
        else:
            seq += line.strip()
    for i in seq_list:
        if i == '':
            seq_list.remove(i)
    return seq_list


In [120]:
# matrix txt reader
def read_matrix(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines


# matrix parser
def parse_matrix(lines):
    matrix = []
    for line in lines:
        if line.startswith(' '):
            item = line.strip().split()
            item.insert(0, ' ')
            matrix.append(item)
        else:
            matrix.append(line.strip().split())

    return matrix

In [121]:
parse_matrix(read_matrix('matrix.txt'))

[[' ', 'A', 'G', 'C', 'T', 'N'],
 ['A', '5', '-1', '-4', '-4', '-2'],
 ['G', '-1', '5', '-4', '-4', '-2'],
 ['C', '-4', '-4', '5', '-1', '-2'],
 ['T', '-4', '-4', '-1', '5', '-2'],
 ['N', '-2', '-2', '-2', '-2', '-1']]

In [122]:
# matrix to dictionary
def matrix_to_dict(matrix):
    matrix_dict = {}
    for i in range(1, len(matrix)):
        for j in range(1, len(matrix[i])):
            matrix_dict[matrix[i][0] + matrix[0][j]] = int(matrix[i][j])
    return matrix_dict

In [123]:
matrix_dict = matrix_to_dict(parse_matrix(read_matrix('matrix.txt')))

In [124]:
close_first_seq = parse_fasta(read_fasta('close-first.fasta'))
close_second_seq = parse_fasta(read_fasta('close-second.fasta'))

In [125]:
def score(term_1, term_2, matrix_dict):
	term = term_1 + term_2
	return matrix_dict[term]

score('A', 'G', matrix_dict)


-1

In [148]:
def needleman_wunsch(seq1, seq2, score_matrix):
    # initialize matrix
    traceback_matrix = np.zeros((len(seq1)+1, len(seq2)+1))
    # fill first row and column
    for i in range(1, len(seq1)+1):
        traceback_matrix[i, 0] = traceback_matrix[i, 0] - i
    for j in range(1, len(seq2)+1):
        traceback_matrix[0, j] = traceback_matrix[0, j] - j
    #print(traceback_matrix)
    # Set up traceback matrix
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match = traceback_matrix[i-1, j-1] + score(seq1[i - 1], seq2[j - 1], score_matrix) + 1
            #mismatch = traceback_matrix[i-1, j-1] + score(seq1[i-1-1], seq2[j-1-1], score_matrix)
            delete = traceback_matrix[i-1,j] - 1
            insert = traceback_matrix[i,j-1] - 1
            traceback_matrix[i, j] = int(max(match, delete, insert))
    # # Traceback
    i = len(seq1)
    j = len(seq2)
    alignment_seq1 = ''
    alignment_seq2 = ''
    while i > 0 and j > 0:
        if traceback_matrix[i, j] == traceback_matrix[i, j - 1] - 1:
            alignment_seq1 = '-' + alignment_seq1
            alignment_seq2 = seq2[j - 1] + alignment_seq2
            j -= 1

        elif traceback_matrix[i, j] == traceback_matrix[i - 1, j] - 1:
            alignment_seq1 = seq1[i - 1] + alignment_seq1
            alignment_seq2 = '-' + alignment_seq2
            i -= 1
        elif traceback_matrix[i, j] == traceback_matrix[i - 1, j -1] + score(
                seq1[i - 1], seq2[j - 1], score_matrix) + 1:
            alignment_seq1 = seq1[i - 1] + alignment_seq1
            alignment_seq2 = seq2[j - 1] + alignment_seq2
            i -= 1
            j -= 1
        else:
            break

    print(alignment_seq1)
    print(alignment_seq2)
    #return traceback_matrix

In [149]:
needleman_wunsch(close_first_seq[0], close_second_seq[0], matrix_dict)

AGACGGAGTTCCT--TCTTGAGGCA-ATCT-GCGTCTC-A-A----TCATAGCCCTCTCTGGCTACCCGCCCGATATCAATC-CTGTTGATTATTT-CACAGCCCACAAGCCGGCCTGA-GCAAG-AACGGA--GCGCCTACCTACTTTTATGATTGGGATTACAGATTACAGACGTTT-GG-TATCTG----AGCC-ATGTCTGCCTA-TAATGATTCGGGCTGGG-CCGAAGTGCTAC-AGTAGACATTAGACATC---C-AAAGCGACGG-TGGATGA-TCTAAATACTTGG
AGACGGAGT-CCTGCTCTTGAGGCACAT-TCGCGTCTCTAGACCCATCATAGCCCTCTCTG-CTACCCGCCCGAT-T-AATCTCTGTTGATTATTTTCACAGCCCACAAGCCGGC-TGAAGCAAGCAACGGACAGCGC-TACCTA-TTTTATG-TTGGGATTACAGATTACAGACGTTTTGGCTATC-GGCCTAGCCCATGTCTGCCTAATA-TGATTCGGGCTGGGACCGAAGTGCT-CCAGTAGACATTAGACATCGGGCCAAAGCGA-GGGTGGA--AGTCTAAATACTTGG


AGACGGAGTTCCT--TCTTGAGGCA-ATCT-GCGTCTC-A-A----TCATAGCCCTCTCTGGCTACCCGCCCGATATCAATC-CTGTTGATTATTT-CACAGCCCACAAGCCGGCCTGA-GCAAG-AACGGA--GCGCCTACCTACTTTTATGATTGGGATTACAGATTACAGACGTTT-GG-TATCTG----AGCC-ATGTCTGCCTA-TAATGATTCGGGCTGGG-CCGAAGTGCTAC-AGTAGACATTAGACATC---C-AAAGCGACGG-TGGATGA-TCTAAATACTTGG

AGACGGAGT-CCTGCTCTTGAGGCACAT-TCGCGTCTCTAGACCCATCATAGCCCTCTCTG-CTACCCGCCCGAT-T-AATCTCTGTTGATTATTTTCACAGCCCACAAGCCGGC-TGAAGCAAGCAACGGACAGCGC-TACCTA-TTTTATG-TTGGGATTACAGATTACAGACGTTTTGGCTATC-GGCCTAGCCCATGTCTGCCTAATA-TGATTCGGGCTGGGACCGAAGTGCT-CCAGTAGACATTAGACATCGGGCCAAAGCGA-GGGTGGA--AGTCTAAATACTTGG

In [ ]:
    # Set up traceback matrix
    for i in range(1, len(seq1)+1):
        for j in range(1, len(seq2)+1):
            match = traceback_matrix[i-1, j-1] + score(seq1[i-1-1], seq2[j-1-1], score_matrix) + 1
            #mismatch = traceback_matrix[i-1, j-1] + score(seq1[i-1-1], seq2[j-1-1], score_matrix)
            delete = traceback_matrix[i-1, j] + score(seq1[i-1-1], seq2[j-1], score_matrix) - 1
            insert = traceback_matrix[i, j-1] + score(seq1[i-1], seq2[j-1-1], score_matrix) - 1
            traceback_matrix[i, j] = max(match, delete, insert)


    # Traceback
    i = len(seq1)
    j = len(seq2)
    alignment_seq1 = ''
    alignment_seq2 = ''
    while i > 0 and j > 0:
        if traceback_matrix[i, j] == traceback_matrix[i-1, j-1] + 1:
            alignment_seq1 = seq1[i-1] + alignment_seq1
            alignment_seq2 = seq2[j-1] + alignment_seq2
            i -= 1
            j -= 1
        elif traceback_matrix[i, j] == traceback_matrix[i-1, j] - 1:
            alignment_seq1 = seq1[i-1] + alignment_seq1
            alignment_seq2 = '-' + alignment_seq2
            i -= 1
        else:
            alignment_seq1 = '-' + alignment_seq1
            alignment_seq2 = seq2[j-1] + alignment_seq2
            j -= 1
    print(alignment_seq1)
    print(alignment_seq2)